### Setup

In [1]:
import sys

if 'google.colab' in sys.modules:
    %pip install -q -U keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


In [2]:
%pip install 'git+https://github.com/jeslago/epftoolbox.git'

  Cloning https://github.com/jeslago/epftoolbox.git to /tmp/pip-req-build-vl9d99o0
  Running command git clone --filter=blob:none --quiet https://github.com/jeslago/epftoolbox.git /tmp/pip-req-build-vl9d99o0
  Resolved https://github.com/jeslago/epftoolbox.git to commit 47d6e0629f65ebd19d3c12cb5689dbad0c2ea078
  Preparing metadata (setup.py) ... done
  Created wheel for epftoolbox: filename=epftoolbox-1.0-py3-none-any.whl size=58746 sha256=87ba3826970452437e36fa9296840bbb91537f34c80746ca610357a0a724e9b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5d5_t4n/wheels/f1/d7/c7/feb15c8f047f5b069656a52f3bcde89a7310a311f9cfac292a
Successfully built epftoolbox


In [3]:
%pip install finta

In [4]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.7 MB/s eta 0:00:00


In [5]:
import os
import sys
from pathlib import Path

import finta
import keras_tuner as kt
import optuna
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import sklearn
import statsmodels
import tensorflow as tf
import tqdm
from google.colab import drive

In [6]:
drive.mount('/content/drive')

DRIVE_ROOT = Path('/content/drive/MyDrive')

PROJECT_ROOT = DRIVE_ROOT / 'projects/btc'
PROJECT_DATA_DIR = PROJECT_ROOT / 'data'
PROJECT_CLEANED_DIR = PROJECT_DATA_DIR / 'cleaned'
PROJECT_EXTERNAL_DIR = PROJECT_DATA_DIR / 'external'

CONFIG_FILENAME = 'config.yaml'
BTC_CLEANED_CSV_FILENAME = 'ohlcv_274_cleaned.csv'
FEDFUNDRATE_CSV_FILENAME = 'fedfunds.csv'
M2SL_CSV_FILENAME = 'm2sl.csv'

CONFIG_FILE_PATH = PROJECT_ROOT / CONFIG_FILENAME
BTC_CLEANED_FILE_PATH = PROJECT_CLEANED_DIR / BTC_CLEANED_CSV_FILENAME
FEDFUNDRATE_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / FEDFUNDRATE_CSV_FILENAME
M2SL_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / M2SL_CSV_FILENAME

Mounted at /content/drive


In [7]:
project_root = os.path.abspath(PROJECT_ROOT)

if project_root not in sys.path:
  sys.path.append(project_root)

print(f'Project root \'{project_root}\' added to sys.path.')

Project root '/content/drive/MyDrive/projects/btc' added to sys.path.


In [8]:
from src.analysis import *
from src.data_loader import *
from src.data_splitter import *
from src.data_cleaner import *
from src.model import *
from src.preprocessing import *
from src.utils import *
from src.visualization import *

In [9]:
print('--- Python Interpreter and Standard Library Versions ---')
print(f'- python: {sys.version}')
print(f'- python info: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

print(f'- os, hashlib, pathlib module: (version tied to Python interpreter)')

print('\n--- Third-Party Library Versions ---')
print(f'- keras_tuner: {kt.__version__}')
print(f'- optuna: {optuna.__version__}')
print(f'- matplotlib: {matplotlib.__version__}')
print(f'- numpy: {np.__version__}')
print(f'- pandas: {pd.__version__}')
print(f'- seaborn: {seaborn.__version__}')
print(f'- sklearn: {sklearn.__version__}')
print(f'- statsmodels: {statsmodels.__version__}')
print(f'- tensorflow: {tf.__version__}')
#print(f'- tqdm: {tqdm.__version__}')

print('\n--- Google Colab Specific Module Versions ---')
print(f'- google.colab.drive: (version tied to Colab environment)')

--- Python Interpreter and Standard Library Versions ---
- python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
- python info: 3.12.12
- os, hashlib, pathlib module: (version tied to Python interpreter)

--- Third-Party Library Versions ---
- keras_tuner: 1.4.7
- optuna: 4.5.0
- matplotlib: 3.10.0
- numpy: 2.0.2
- pandas: 2.2.2
- seaborn: 0.13.2
- sklearn: 1.6.1
- statsmodels: 0.14.5
- tensorflow: 2.19.0

--- Google Colab Specific Module Versions ---
- google.colab.drive: (version tied to Colab environment)


In [10]:
%pip show epftoolbox

Name: epftoolbox
Version: 1.0
Summary: An open-access benchmark and toolbox for electricity price forecasting
Home-page: https://github.com/jeslago/epftoolbox
Author: Jesus Lago
Author-email: jesuslagogarcia@gmail.com
License: GNU AGPLv3
Location: /usr/local/lib/python3.12/dist-packages
Requires: hyperopt, keras, matplotlib, numpy, pandas, scikit-learn, scipy, statsmodels, tensorflow
Required-by: 


In [11]:
%pip show finta

Name: finta
Version: 1.3
Summary: Common financial technical indicators implemented in Pandas.
Home-page: https://github.com/peerchemist/finta
Author: Peerchemist
Author-email: peerchemist@protonmail.ch
License: LGPLv3+
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, pandas
Required-by: 


### Load the cleaned dataset

In [12]:
cleaned_ds = load_btc_ds(BTC_CLEANED_FILE_PATH, 'date', True)


Loading dataset from: ohlcv_274_cleaned.csv...
Dataset loaded successfully.


### Perform linear interpolation to fill the missing values on March 15, 2025 to ensure a continuous time series for the ARIMA model.

In [13]:
imputed_ds = fill_time_series_gaps(cleaned_ds, 'min')

### Resample the cleaned dataset into multiple timeframes.

In [14]:
resampled_ds = resample_btc_data(imputed_ds)


--- Resampling BTC Data to Multiple Timeframes ---
- Resampling to hourly frequency...
- Resampling to daily frequency...
- Resampling to weekly frequency...
- Resampling to monthly frequency...
--- BTC Data Resampling Complete ---


### Split the resampled datasets into training, validation, and test sets of the target timeframe.

In [15]:
train_ds, valid_ds, test_ds = split_btc_ds(resampled_ds['daily'], 'Day')


--- Splitting the BTC Dataset into Training, Validation, and Test Sets (Day) ---
--- BTC Dataset Splitting Complete ---


### Run the ADF test on datasets at multiple frequencies to verify stationarity and seasonality

In [17]:
run_adf_test(calculate_log_and_diff(train_ds.copy(), 'close'), 'log_returns_close', 'BTC Daily Closing Returns')

= ADF Test: BTC Daily Closing Returns =
ADF Statistics: -10.8445
p-value: 0.0000
Critical Values:
	1%:-3.4320
	5%:-2.8623
	10%:-2.5672
Conclusion: The p-value is less than or equal to 0.05. The data is likely stationary and seasonal.



### Prepare the dataset by consolidating data splits,adding a comprehensive suite of features, and then splitting the data back.

In [18]:
train_prep_ds, valid_prep_ds, test_prep_ds = prepare_feature_ds(
    train_ds, valid_ds, test_ds,
    ['open', 'high', 'low', 'close', 'volume'],
    FEDFUNDRATE_CSV_FILE_PATH, M2SL_CSV_FILE_PATH
)

= Feature Engineering and Dataset Preparation =
Step 1: Concatenating data splits...
- Combined train, validation, and test sets into a single DataFrame. Full shape: (4922, 5)

Step 2: Engineering new features...
- Calculated log returns for ['open', 'high', 'low', 'close', 'volume'].
- Calculated the high-low price range as a percentage of the open price.
- Calculated Simple Moving Average (SMA) for trend.
- Calculated Relative Strength Index (RSI) for momentum.
- Calculated Average True Range (ATR) for volatility.
- Calculated On-Balance Volume (OBV) for volume pressure.
- Calculated Moving Average Convergence Divergence (MACD).
- Engineered relationship features
- Calculated technical indicators.


/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


- Macro data loaded and prepared.
- Macro data merged.
- Added macroeconomic features.

Step 3: Handling missing and infinite values...
- Replaced 0 infinite values (inf, -inf) with NaN.
- Forward-filled missing values to maintain data continuity.
- Shape before dropping NaNs: (4922, 28)
- Shape after dropping NaNs: (4873, 28)
- Dropped initial rows containing NaNs that resulted from feature calculations.

Step 4: Splitting into final datasets...
- Split the single processed DataFrame back into train, validation, and test sets.
- Train shape: (3888, 28)
- Valid shape: (492, 28)
- Test shape: (493, 28)
= Feature Preparation Complete =


### Time-Series decomposition and Autocorrelation Analysis
*   Visualize seasonal patterns by decomposing the time series into its trend, seasonal, and residual components.

*   Analyze the raw closing price data, aggregated at daily frequencies, to detect corresponding weekly, and monthly seasonality.

*    Apply a log-difference transformation to the series to achieve stationarity, then test for any residual seasonality.

*    Plot the Autocorrelation (ACF) and Partial Autocorrelation (PACF) of the log-differenced series to assess stationarity and determine if further differencing is necessary.

In [20]:
plot_time_series_decomposition(train_prep_ds, 'close', 'multiplicative', 7, 'BTC Weekly Seasonality (Closing Price - Daily Resampled)', 'btc_weekly_seasonality_daily_price_decomposition.png')
plot_time_series_decomposition(train_prep_ds, 'close', 'multiplicative', 30, 'BTC Monthly Seasonality (Closing Price - Daily Resampled)', 'btc_monthly_seasonality_daily_price_decomposition.png')

plot_time_series_decomposition(train_prep_ds, 'log_returns_close', 'additive', 7, 'BTC Weekly Seasonality (Closing Log Returns) - Daily Resampled)', 'btc_weekly_seasonality_daily_log_returns_decomposition.png' )
plot_time_series_decomposition(train_prep_ds, 'log_returns_close', 'additive', 30, 'BTC Monthly Seasonality (Closing Log Returns) - Daily Resampled)', 'btc_monthly_seasonality_daily_log_returns_decomposition.png' )

plot_autocorrelation(train_prep_ds, 'log_returns_close', 60, 'BTC Daily Closing Log Returns', 'daily_log_returns_autocorrelation')

Plot saved to 'outputs/plots/btc_weekly_seasonality_daily_price_decomposition.png'.
Plot saved to 'outputs/plots/btc_monthly_seasonality_daily_price_decomposition.png'.
Plot saved to 'outputs/plots/btc_weekly_seasonality_daily_log_returns_decomposition.png'.
Plot saved to 'outputs/plots/btc_monthly_seasonality_daily_log_returns_decomposition.png'.
Plot saved to 'outputs/plots/daily_log_returns_autocorrelation_autocorrelation.png'.


### ARIMA Model Tuning (1-Week)

In [ ]:
WINDOW_SIZE = 7
FORECAST_HORIZON = 7
TARGET_COL = 'close'

In [ ]:
naive_metrics, naive_pred_out = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 7, 'Validation', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Validation Set (Horizon: 7 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 4.7134%
- Directional Accuracy (DA): 44.7699%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-02-22: $52003.00
- Directional Signal for 2024-02-22: 1.0


In [ ]:
p_values = range(0, 7)
d_values = [1]
q_values = range(0, 7)

arima_order_and_metrics = find_best_arima_order(train_prep_ds, valid_prep_ds, TARGET_COL,
                                                WINDOW_SIZE, FORECAST_HORIZON, 'Days',
                                                p_values, d_values, q_values, naive_metrics)

= ARIMA Model Tuning on Validation Set (Horizon: 7 Days) =
Starting ARIMA grid search for 49 combinations...



ARIMA Grid Search:   2%|▏         | 1/49 [00:02<01:59,  2.49s/it]

 Order: (0, 1, 0), MAPE: 5.0698%, DA: 50.3093%


ARIMA Grid Search:   4%|▍         | 2/49 [00:09<04:05,  5.23s/it]

 Order: (0, 1, 1), MAPE: 5.0718%, DA: 50.7216%


...

 Order: (6, 1, 4), MAPE: 5.0934%, DA: 51.5464%


ARIMA Grid Search:  98%|█████████▊| 48/49 [08:51<00:16, 16.05s/it]

 Order: (6, 1, 5), MAPE: 5.1059%, DA: 50.7216%


ARIMA Grid Search: 100%|██████████| 49/49 [09:12<00:00, 11.27s/it]

 Order: (6, 1, 6), MAPE: 5.1282%, DA: 50.1031%

ARIMA grid search complete.

--- Selecting Best ARIMA Model ---
- Naive Model Benchmark → MAPE: 4.7134%, DA: 44.7699%

- Conclusion: No ARIMA model outperformed the naive model.


In [ ]:
arima_order_and_metrics

({'order': (5, 1, 3),
  'mape': 5.099797667444847,
  'da': np.float64(51.75257731958764),
  'comp':                true          pred
  2022-10-19  19226.0  18938.550607
  2022-10-20  19061.0  19147.112475
  2022-10-21  19189.0  19326.413185
  2022-10-22  19186.0  19108.103868
  2022-10-23  19507.0  19121.986718
  ...             ...           ...
  2024-02-11  48295.0  43053.509232
  2024-02-12  49745.0  42695.989632
  2024-02-13  49126.0  42620.192475
  2024-02-14  51475.0  43212.713726
  2024-02-15  52003.0  43589.706743
  
  [485 rows x 2 columns]},
 False)

In [ ]:
metrics, beats_baseline = arima_order_and_metrics

In [ ]:
metrics

{'order': (5, 1, 3),
 'mape': 5.099797667444847,
 'da': np.float64(51.75257731958764),
 'comp':                true          pred
 2022-10-19  19226.0  18938.550607
 2022-10-20  19061.0  19147.112475
 2022-10-21  19189.0  19326.413185
 2022-10-22  19186.0  19108.103868
 2022-10-23  19507.0  19121.986718
 ...             ...           ...
 2024-02-11  48295.0  43053.509232
 2024-02-12  49745.0  42695.989632
 2024-02-13  49126.0  42620.192475
 2024-02-14  51475.0  43212.713726
 2024-02-15  52003.0  43589.706743
 
 [485 rows x 2 columns]}

In [ ]:
plot_residuals_analysis(metrics, 'comp', 14, 'ARIMA', 'Week')
run_ljung_box_test(metrics, 'comp', 14, 'ARIMA')
print(metrics['comp'])

Plot saved to 'outputs/plots/arima_residuals_analysis_week'.
= Ljung-Box Test: ARIMA =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true          pred
2022-10-19  19226.0  18938.550607
2022-10-20  19061.0  19147.112475
2022-10-21  19189.0  19326.413185
2022-10-22  19186.0  19108.103868
2022-10-23  19507.0  19121.986718
...             ...           ...
2024-02-11  48295.0  43053.509232
2024-02-12  49745.0  42695.989632
2024-02-13  49126.0  42620.192475
2024-02-14  51475.0  43212.713726
2024-02-15  52003.0  43589.706743

[485 rows x 2 columns]


### ARIMA Model Tuning (1-Month)

In [ ]:
WINDOW_SIZE = 30
FORECAST_HORIZON = 30
TARGET_COL = 'close'

In [ ]:
naive_metrics, naive_pred = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 30, 'Validation', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Validation Set (Horizon: 30 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 10.5813%
- Directional Accuracy (DA): 54.8611%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-03-16: $52003.00
- Directional Signal for 2024-03-16: 1.0


In [ ]:
p_values = range(0, 7)
d_values = [1]
q_values = range(0, 7)

arima_order_and_metrics = find_best_arima_order(train_prep_ds, valid_prep_ds, TARGET_COL,
                                                WINDOW_SIZE, FORECAST_HORIZON, 'Days',
                                                p_values, d_values, q_values, naive_metrics)

= ARIMA Model Tuning on Validation Set (Horizon: 30 Days) =
Starting ARIMA grid search for 49 combinations...



ARIMA Grid Search:   2%|▏         | 1/49 [00:02<02:14,  2.79s/it]

 Order: (0, 1, 0), MAPE: 10.8234%, DA: 46.9697%


ARIMA Grid Search:   4%|▍         | 2/49 [00:06<02:44,  3.49s/it]

 Order: (0, 1, 1), MAPE: 10.8244%, DA: 47.4026%


...

 Order: (6, 1, 4), MAPE: 10.8226%, DA: 46.5368%


ARIMA Grid Search:  98%|█████████▊| 48/49 [09:25<00:18, 18.01s/it]

 Order: (6, 1, 5), MAPE: 10.8176%, DA: 46.9697%


ARIMA Grid Search: 100%|██████████| 49/49 [09:45<00:00, 11.94s/it]

 Order: (6, 1, 6), MAPE: 10.8105%, DA: 48.7013%

ARIMA grid search complete.

--- Selecting Best ARIMA Model ---
- Naive Model Benchmark → MAPE: 10.5813%, DA: 54.8611%

- Conclusion: No ARIMA model outperformed the naive model.


In [ ]:
metrics, beats_baseline = arima_order_and_metrics

In [ ]:
metrics

{'order': (5, 1, 6),
 'mape': 11.728842179995596,
 'da': np.float64(52.16450216450217),
 'comp':                true          pred
 2022-11-11  16806.0  18016.865973
 2022-11-12  16842.0  18848.171907
 2022-11-13  16499.0  19201.511769
 2022-11-14  16590.0  18883.152226
 2022-11-15  16721.0  18889.004922
 ...             ...           ...
 2024-02-11  48295.0  48630.140966
 2024-02-12  49745.0  43827.026508
 2024-02-13  49126.0  43096.635634
 2024-02-14  51475.0  42858.865378
 2024-02-15  52003.0  42953.541014
 
 [462 rows x 2 columns]}

In [ ]:
plot_residuals_analysis(metrics, 'comp', 90, 'ARIMA', 'Month')
run_ljung_box_test(metrics, 'comp', 90, 'ARIMA')
print(metrics['comp'])

Plot saved to 'outputs/plots/arima_residuals_analysis_month'.
= Ljung-Box Test: ARIMA =
Lags tested: 90
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true          pred
2022-11-11  16806.0  18016.865973
2022-11-12  16842.0  18848.171907
2022-11-13  16499.0  19201.511769
2022-11-14  16590.0  18883.152226
2022-11-15  16721.0  18889.004922
...             ...           ...
2024-02-11  48295.0  48630.140966
2024-02-12  49745.0  43827.026508
2024-02-13  49126.0  43096.635634
2024-02-14  51475.0  42858.865378
2024-02-15  52003.0  42953.541014

[462 rows x 2 columns]


### Save hyperparameters to a YAML file.

In [ ]:
orders_to_save = {
  'ARIMA': {
        'weekly': {
            'order': [5, 1, 3]
        },
        'monthly': {
            'order': [5, 1, 6]
        },
    }
}

In [ ]:
save_config(orders_to_save, CONFIG_FILE_PATH)

Configuration successfully saved to '/content/drive/MyDrive/projects/btc/config.yaml'.
